In [3]:
import json
import graphene

from __future__ import print_function


In [6]:
class TrialArgsData(graphene.InputObjectType):
    name = 'TrialArgsData'
    description = 'For the celery task: engine:train:get-trial-result'

    dataset = graphene.String()
    modelchoice = graphene.String()
    project = graphene.String()
    train_id = graphene.String()
    transfo_id = graphene.String()

In [7]:
class TrialArgs(graphene.InputObjectType):
    meta=graphene.String()
    data=TrialArgsData

In [8]:
class TrialResults(graphene.InputObjectType):
    task = graphene.String(required=True)
    kwargs = graphene.String(required=True)
    id = graphene.String(required=True)  # TODO: rename id_
    args = TrialArgs(required=True)

In [9]:
class TrialTypeDefinition(graphene.ObjectType):
    task = graphene.String()
    kwargs = graphene.String()
    id = graphene.String()
    args = TrialArgs

In [10]:
class CreateTrialMutation(graphene.Mutation):

    class Arguments:
        # trial_results will be replaced by
        # trialResults in graphQL format
        trial_results = TrialResults(required=True)

    Output = TrialTypeDefinition

    def mutate(self, info, trial_results):
        return TrialTypeDefinition(
            task=trial_results.task,
            kwargs=trial_results.kwargs,
            id=trial_results.id,
            args=trial_results.args
        )

In [11]:
class TrialMutation(graphene.ObjectType):
    # create_trial_mutation will be replaced by
    # createTrialMutation
    # in graphQL format
    create_trial_mutation = CreateTrialMutation.Field()

In [12]:
schema = graphene.Schema(mutation=TrialMutation)


mutate_args = '''
task:\"engine:train:get-trial-result\",
kwargs:\"\",
id:\"ed0f9e10-7b43-42da-93b4-7b3e0f0fb70f\"'''

hard_coded_mutation = '''
    mutation toto{
      createTrialMutation(trialResults: { ''' + mutate_args + '''}) {
        task
        kwargs
        id
      }
    }
'''

In [13]:
def validateTrialMutation(json_data):
    print("In get_trial_mutation: validateTrialMutation")

    # first test with a hard-coded
    # graphQL mutation
    graphQL_muta = hard_coded_mutation
    print(graphQL_muta)

    print("<---JSON DATA ---->\n", json_data)

    # translate json data to GraphQL format
    # TODO: write the adapter function

    # muta1
    result1 = schema.execute(graphQL_muta)
    print("<---Results 1 ---->\n", result1)
    print("<---Results 1 / data ---->\n", result1.data)
    # print(result1.data['createAddress']['latlng'])
    print("<---ERRORS ---->\n", result1.errors)
    raw_input("muta1 executed\n")

    # validatFromSchema(schema, data_to_validate)
    # print(CreateTrialMutation.Field())
    # if 'task' not in data_block:
    #     return False

    return True

In [16]:
    print("Please provide json file name")
    json_file = "trial_results.json"  # raw_input(">> ")
    # Load the json in a dirctionnary
    data = json.load(open(json_file))

    print(data.keys())

    # Now we check if json stored in data
    # is compliant with get-trial-results schema
    validateTrialMutation(data)

Please provide json file name
[u'task', u'id', u'args', u'kwargs']
In get_trial_mutation: validateTrialMutation

    mutation toto{
      createTrialMutation(trialResults: { 
task:"engine:train:get-trial-result",
kwargs:"",
id:"ed0f9e10-7b43-42da-93b4-7b3e0f0fb70f"}) {
        task
        kwargs
        id
      }
    }

<---JSON DATA ---->
 {u'task': u'engine:train:get-trial-result', u'id': u'ed0f9e10-7b43-42da-93b4-7b3e0f0fb70f', u'args': [{u'status': u'requested'}, {u'project': u'5ac7a6edf644b20166caf596', u'modelchoice': u'supervised', u'transfo_id': u'5ace094a761e68003c34fda0', u'train_id': u'5ace094a761e68003c34fda1', u'dataset': u'5ac7a6faf644b20166caf597'}], u'kwargs': {}}
<---Results 1 ---->
<---Results 1 / data ---->
 None
<---ERRORS ---->
 [GraphQLError('Argument "trialResults" has invalid value {task: "engine:train:get-trial-result", kwargs: "", id: "ed0f9e10-7b43-42da-93b4-7b3e0f0fb70f"}.\nIn field "args": Expected "TrialArgs!", found null.',)]
muta1 executed



True